In [ ]:
# リスト 3-5-1 Wikipediaの日本百名湯記事で類似文書検索（説明コメント付き）
# --------------------------------------------------------------------------------
# 目的:
#   ・Wikipedia の日本百名湯（記事あり）の各ページ本文を収集し、後続のベクトル化（TF-IDF など）や
#     類似文書検索（コサイン類似度など）の入力とするためのデータ構造（data_list）を構築する。
#
# 収集方針（理論/設計）:
#   ・タイトルは Wikipedia のページ見出しに「完全一致」させる（曖昧さ回避を避けるため）。
#   ・wikipedia パッケージの `page(title, auto_suggest=False)` を用いることで、
#     自動補完による誤ジャンプを抑止し、表記ゆれに強すぎる検索を避ける。
#   ・得られる `.content` は本文テキスト（マークアップはある程度除去済み）で、節見出し等は含まれる。
#   ・結果は辞書の配列 data_list に格納する（id/title/text の 3 キー）。`app_id` は 1 始まりの連番。
#
# 実務上の注意:
#   ・大量アクセスは先方に負荷となるため、必要に応じて `time.sleep` を入れる（礼儀的なレート制御）。
#   ・曖昧さ回避/ページ欠落などの例外（DisambiguationError, PageError）を適切にハンドリングする。
#   ・後続処理（形態素解析・ベクトル化）での再現性確保のため、成功/失敗リストをロギングするのが望ましい。
#   ・本文の前処理（見出し/注記の除去、正規化、句読点処理など）は別段階で行うと責務分離が明確。
# --------------------------------------------------------------------------------

# 日本百名湯の記事タイトル一覧（Wikipedia のページ見出しに合わせた表記）
title_list = [
    "菅野温泉",
    "養老牛温泉",
    "定山渓温泉",
    "登別温泉",
    "洞爺湖温泉",
    "ニセコ温泉郷",
    "朝日温泉 (北海道)",
    "酸ヶ湯温泉",
    "蔦温泉",
    "花巻南温泉峡",
    "夏油温泉",
    "須川高原温泉",
    "鳴子温泉郷",
    "遠刈田温泉",
    "峩々温泉",
    "乳頭温泉郷",
    "後生掛温泉",
    "玉川温泉 (秋田県)",
    "秋ノ宮温泉郷",
    "銀山温泉",
    "瀬見温泉",
    "赤倉温泉 (山形県)",
    "東山温泉",
    "飯坂温泉",
    "二岐温泉",
    "那須温泉郷",
    "塩原温泉郷",
    "鬼怒川温泉",
    "奥鬼怒温泉郷",
    "草津温泉",
    "伊香保温泉",
    "四万温泉",
    "法師温泉",
    "箱根温泉",
    "湯河原温泉",
    "越後湯沢温泉",
    "松之山温泉",
    "大牧温泉",
    "山中温泉",
    "山代温泉",
    "粟津温泉",
    "奈良田温泉",
    "西山温泉 (山梨県)",
    "野沢温泉",
    "湯田中温泉",
    "別所温泉",
    "中房温泉",
    "白骨温泉",
    "小谷温泉",
    "下呂温泉",
    "福地温泉",
    "熱海温泉",
    "伊東温泉",
    "修善寺温泉",
    "湯谷温泉 (愛知県)",
    "榊原温泉",
    "木津温泉",
    "有馬温泉",
    "城崎温泉",
    "湯村温泉 (兵庫県)",
    "十津川温泉",
    "南紀白浜温泉",
    "南紀勝浦温泉",
    "湯の峰温泉",
    "龍神温泉",
    "奥津温泉",
    "湯原温泉",
    "三朝温泉",
    "岩井温泉",
    "関金温泉",
    "玉造温泉",
    "有福温泉",
    "温泉津温泉",
    "湯田温泉",
    "長門湯本温泉",
    "祖谷温泉",
    "道後温泉",
    "二日市温泉 (筑紫野市)",
    "嬉野温泉",
    "武雄温泉",
    "雲仙温泉",
    "小浜温泉",
    "黒川温泉",
    "地獄温泉",
    "垂玉温泉",
    "杖立温泉",
    "日奈久温泉",
    "鉄輪温泉",
    "明礬温泉",
    "由布院温泉",
    "川底温泉",
    "長湯温泉",
    "京町温泉",
    "指宿温泉",
    "霧島温泉郷",
    "新川渓谷温泉郷",
    "栗野岳温泉",
]

# Wikipedia から本文を読み取る
# - `wikipedia` ライブラリは MediaWiki API を叩く簡易ラッパ。オンライン接続が必要。
import wikipedia

wikipedia.set_lang("ja")  # 日本語版 Wikipedia に固定（他言語へ飛ぶのを防ぐ）

data_list = []  # 後続の類似検索・ベクトル化に渡すための原データを蓄積するリスト
for index, title in enumerate(title_list):
    # 進捗確認のため、1 始まりの連番とタイトルを表示（学習/デバッグ用途）
    print(index + 1, title)

    # auto_suggest=False:
    #  - 与えたタイトルでの厳密取得を志向。誤補完により別ページへ飛ぶことを防ぐ。
    #  - 表記が完全一致していないと例外（PageError 等）になる可能性がある点に留意。
    text = wikipedia.page(title, auto_suggest=False).content

    # 構造化: app_id（連番）, title（記事名）, text（本文）という素朴なレコード形式
    item = {
        "app_id": index
        + 1,  # 1, 2, 3, ...（検索結果表示や ES の _id 等に流用しやすい）
        "title": title,  # 記事タイトル（後続での見出し・ログ用）
        "text": text,  # 記事本文（前処理→ベクトル化の入力）
    }
    data_list.append(item)

# --------------------------------------------------------------------------------
# （拡張: 例外処理 + レート制御の雛形。必要に応じて使用）
# import time
# from wikipedia.exceptions import DisambiguationError, PageError
# failed = []
# data_list = []
# for index, title in enumerate(title_list):
#     print(index + 1, title)
#     try:
#         page = wikipedia.page(title, auto_suggest=False)
#         data_list.append({'app_id': index + 1, 'title': title, 'text': page.content})
#         time.sleep(0.5)  # マナー的ウェイト。必要に応じて調整
#     except DisambiguationError as e:
#         failed.append({'title': title, 'reason': 'disambiguation', 'options': e.options[:5]})
#     except PageError:
#         failed.append({'title': title, 'reason': 'page_not_found'})
# # 収集サマリを確認（学習用）
# # print(f"成功: {len(data_list)} / 失敗: {len(failed)}"); print(failed)
# --------------------------------------------------------------------------------

In [ ]:
# Elasticsearchインスタンスの生成
# 3.3節参照

from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
# インデックス作成用JSONの定義（詳細コメント付き）
# 3.3節参照
# -----------------------------------------------------------------------------
# 目的:
#   ・Elasticsearch に日本語向けの analysis 設定（正規化・形態素解析・同義語等）を与え、
#     検索時/索引用で異なるアナライザを使い分ける土台を作る。
#
# 理論ポイント:
#   ・アナライザは「char_filter → tokenizer → filter」の順で適用される。
#   ・索引時（document投入）: analyzer="jpn-index" を適用し、語彙を“素直に”保存。
#   ・検索時（クエリ解析） : search_analyzer="jpn-search" を適用し、同義語などで想起を拡張。
#     → 索引側に同義語を混ぜないことで観測性と保守性が高まるのが定石。
#
# 実務メモ:
#   ・icu_normalizer を使うには analysis-icu プラグインが必要。
#   ・kuromoji_* を使うには analysis-kuromoji プラグインが必要（バージョンにより同梱/外部）。
#   ・"user_dictionary" のパスは **Elasticsearchノード上** の配置先（ローカル開発機ではない）。
#   ・同義語タイプ "synonym" は等価展開。片方向正規化や位置情報重視には "synonym_graph" も検討。
# -----------------------------------------------------------------------------

create_index = {
    "settings": {
        "analysis": {
            # ---------------------------
            # Token Filter 群（語彙変換）
            # ---------------------------
            "filter": {
                "synonyms_filter": {  # 同義語フィルタの定義（検索側で使用予定）
                    "type": "synonym",
                    "synonyms": [  # 等価展開ルール群（ここでは空/プレースホルダ）
                        # 例: "すし,スシ,鮨,寿司"
                    ],
                }
            },
            # ---------------------------
            # Tokenizer（形態素分割）
            # ---------------------------
            "tokenizer": {
                "kuromoji_w_dic": {  # ユーザー辞書付きの kuromoji トークナイザ
                    "type": "kuromoji_tokenizer",  # ※ 綴りは "kuromoji"
                    "user_dictionary": "my_jisho.dic",  # ESノード側パス（*.dic=事前ビルド済み）
                }
            },
            # ---------------------------
            # Analyzer（前処理パイプライン）
            # ---------------------------
            "analyzer": {
                # 検索用: 想起重視（同義語をここで展開）
                "jpn-search": {
                    "type": "custom",
                    "char_filter": [
                        "icu_normalizer",  # 文字正規化（NFKC等）: 全半角/互換文字の吸収
                        "kuromoji_iteration_mark",  # 繰り返し記号（々/ゝ等）の展開
                    ],
                    "tokenizer": "kuromoji_w_dic",  # 形態素解析（ユーザー辞書適用）
                    "filter": [
                        "synonyms_filter",  # ← 同義語展開（検索側のみ）
                        "kuromoji_baseform",  # 活用の原形化（形容詞/動詞など）
                        "kuromoji_part_of_speech",  # 不要品詞の除去（助詞・助動詞・記号 等）
                        "ja_stop",  # 日本語ストップワード除去
                        "kuromoji_number",  # 数字正規化（漢数字→算用 等）
                        "kuromoji_stemmer",  # 長音正規化（コンピューター→コンピュータ）
                    ],
                },
                # 索引用: 観測性重視（同義語は含めない）
                "jpn-index": {
                    "type": "custom",
                    "char_filter": ["icu_normalizer", "kuromoji_iteration_mark"],
                    "tokenizer": "kuromoji_w_dic",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "ja_stop",
                        "kuromoji_number",
                        "kuromoji_stemmer",
                    ],
                },
            },
        }
    }
}

# 日本語用インデックス名の定義
jp_index = "jp_index"

# 既存インデックスがあれば削除（学習/検証用途）
# 本番運用では: 新インデックス作成 → _reindex 移行 → エイリアス切替 が安全
if es.indices.exists(index=jp_index):
    es.indices.delete(index=jp_index)

# インデックスの作成（settings のみ）
# 注意: この段階ではフィールドへの analyzer 割当は未定義。
#       → 後続で put_mapping し、"content" などの text フィールドに
#          analyzer="jpn-index" / search_analyzer="jpn-search" を明示すること。
es.indices.create(index=jp_index, body=create_index)

# ----（参考: 動作確認の雛形。必要時のみコメント解除）-------------------------
# print(es.indices.analyze(index=jp_index, body={"analyzer":"jpn-search","text":"寿司"}))
# mapping = {"properties":{"content":{"type":"text","analyzer":"jpn-index","search_analyzer":"jpn-search"}}}
# es.indices.put_mapping(index=jp_index, body=mapping)
# -----------------------------------------------------------------------------

In [ ]:
# mappingの設定（詳細コメント付き）
# 3.3節参照
# -----------------------------------------------------------------------------
# 目的:
#   ・インデックス `jp_index` に対して、全文検索対象フィールドのマッピングを設定する。
#   ・索引時は「jpn-index」、検索時は「jpn-search」を適用して役割分離する
#     （= 索引は素直に、検索で想起拡張：同義語など）。
#
# 理論メモ:
#   ・Elasticsearch では analyzer は
#       char_filter → tokenizer → filter
#     の順で適用される。
#   ・`analyzer` は索引時（ドキュメント投入時）に使用、
#     `search_analyzer` は検索クエリ解析時に使用される。
#   ・既存フィールドの analyzer は **後から変更できない**（型制約）。
#     変更が必要な場合は「新インデックス作成 → _reindex → エイリアス切替」が必要。
#
# 実務メモ:
#   ・title の完全一致・集計・ソート用に keyword のサブフィールド（raw）を併設するのが定石。
#   ・`jpn-search` に同義語を入れている場合、索引側に同義語を焼き込まないことで
#     可観測性と運用保守性が高くなる（デバッグ時に“生の語彙”が見える）。
# -----------------------------------------------------------------------------

mapping = {
    "properties": {
        "text": {
            "type": "text",
            # 索引時に適用: 形態素解析・正規化は行うが、同義語はここでは展開しない
            "analyzer": "jpn-index",
            # 検索時に適用: 同義語や原形化などで表記ゆれを吸収
            "search_analyzer": "jpn-search",
        },
        "title": {
            "type": "text",
            "analyzer": "jpn-index",
            "search_analyzer": "jpn-search",
            # 完全一致/集計/ソート向けの keyword サブフィールド
            "fields": {
                "raw": {
                    "type": "keyword",
                    "ignore_above": 256,
                    # 正規化を厳密にしたい場合は settings 側に normalizer を定義して指定する
                    # 例: "normalizer": "lowercase_normalizer"
                }
            },
        },
    }
}

# マッピング適用
es.indices.put_mapping(index=jp_index, body=mapping)

# --- 任意: 適用結果の確認（観測性向上のための簡易ダンプ） --------------------
# cur = es.indices.get_mapping(index=jp_index)
# from pprint import pprint
# pprint(cur[jp_index]["mappings"]["properties"])
# -----------------------------------------------------------------------------

# 参考（比較用・非推奨パターン）:
#   analyzer="jpn-search" を索引時にも使うと、同義語が索引側に焼き込まれ、
#   後から同義語辞書を差し替えた際の検証が難しくなるため運用上は避けるのが無難。
#   ただし “検索時と索引時を完全同一にしたい” という方針なら一貫性は高い（可観測性は低下）。

In [ ]:
# 文書の登録（詳細コメント付き）
# 3.3節参照
# -----------------------------------------------------------------------------
# 目的:
#   ・前段で収集した Wikipedia 記事データ（data_list）を、インデックス jp_index に投入する。
#   ・各レコードの 'app_id' を Elasticsearch の _id として用い、後続の類似検索時に
#     クエリ元と結果の対応を取りやすくする（例: self-match 除外や ID ベースの参照に便利）。
#
# 前提:
#   ・`jp_index` は settings（日本語アナライザ）と mappings（text/title フィールド）を設定済み。
#   ・`data_list` は以下の辞書要素を持つ配列:
#       {'app_id': int, 'title': str, 'text': str}
#   ・アナライザ分離方針（索引: jpn-index / 検索: jpn-search）により、
#     索引側は“素直な語彙”を保存、検索側で同義語等を展開する。
#
# 実務メモ:
#   ・このループは 1 件ずつ HTTP リクエストを発行するため、大量投入時は遅くなる。
#     → 大規模データでは helpers.bulk の利用を推奨（下に雛形）。
#   ・直後に検索して可視化したい場合は refresh="wait_for" を付けるか、事後に refresh API を呼ぶ。
#   ・同一 _id の再投入は上書き（_version が上がる）。衝突で失敗させたい場合は op_type="create" を指定。
# -----------------------------------------------------------------------------

for body in data_list:
    # id と app_id の値を同じにして、類似検索をやりやすくする
    # - 後続で「ある文書に最も近い文書」を探す際、_id をキーに自己一致を除外/識別しやすい。
    es.index(index=jp_index, id=body["app_id"], body=body)
    # 参考（8.x 推奨の引数名。上書き禁止・即時可視化の例）:
    # es.index(index=jp_index, id=body['app_id'], document=body, op_type="create", refresh="wait_for")

# 直後に検索評価を行う場合の明示リフレッシュ（refresh="wait_for" を使わない場合の代替）
# es.indices.refresh(index=jp_index)

# -----------------------------------------------------------------------------
# 参考: bulk で高速投入する雛形（大量データ向け）
# -----------------------------------------------------------------------------
# from elasticsearch import helpers
# actions = (
#     {
#         "_op_type": "index",      # "create" にすると既存IDはエラー
#         "_index": jp_index,
#         "_id": doc["app_id"],
#         "_source": doc
#     }
#     for doc in data_list
# )
# helpers.bulk(es, actions, refresh="wait_for")  # 成功/失敗件数を返す
# -----------------------------------------------------------------------------

In [ ]:
# リスト 3.5.2 類似検索の実行（詳細コメント付き）
# -----------------------------------------------------------------------------
# 目的:
#   ・Elasticsearch の More Like This（MLT）クエリで、指定の文書（ここでは _id=3: 定山渓温泉）
#     と“内容的に似ている”文書を検索する。
#
# 理論メモ（MLT の挙動）:
#   ・MLT は「参照文書（like）」の本文から“代表語（terms）”を抽出し、クエリを自動生成する。
#     抽出語はフィールドのアナライザで解析される（ここでは text フィールドの search_analyzer が既定）。
#   ・スコアリングは基本的に BM25。共通語（df が高い語）は IDF が小さくなり寄与が下がる。
#   ・`min_term_freq`, `min_doc_freq`, `max_query_terms`, `minimum_should_match` などで
#     「どの程度の珍しさ/一致度」を要求するかを制御できる。
#   ・デフォルトでは `include=false` で参照文書そのものは結果から除外される（自己一致の抑制）。
#
# 実務メモ:
#   ・Elasticsearch 7 以降は type 廃止（_type を指定しないのが基本）。8 系では完全削除。
#     → 互換性重視なら like に _type は含めない（下に修正例を併記）。
#   ・直前に index したデータを即座に検索する場合は refresh を考慮（refresh="wait_for" など）。
#   ・日本語の場合、検索側アナライザ（ここでは jpn-search）で同義語/正規化を効かせる設計が有効。
# -----------------------------------------------------------------------------

# 検索条件の設定
# - 元コード（_type を含む; 旧バージョン互換）
query = {
    "query": {
        "more_like_this": {
            "fields": ["text"],  # 類似度の計算に使うフィールド
            "like": [
                {
                    "_index": "jp_index",
                    "_type": "_doc",  # ← 7+ では非推奨/8 では無効。可能なら削除を推奨
                    "_id": "3",  # 参照文書（app_id と _id を一致させてある前提）
                }
            ],
            # 代表的なチューニング項目（必要に応じて追加）
            # ,"min_term_freq": 1          # 参照文書内での最低出現回数（文書内ストップ語化）
            # ,"min_doc_freq": 1           # コーパス内での最低文書頻度（汎用語の除外）
            # ,"max_query_terms": 50       # クエリ化する語の最大数（既定: 25）
            # ,"minimum_should_match": "30%"  # 一致率をパーセンテージ指定で要求
            # ,"analyzer": "jpn-search"    # 明示したい場合のみ（既定はフィールドの search_analyzer）
            # ,"stop_words": ["温泉"]      # ドメイン特有の汎用語を明示的に無視
            # ,"include": False            # 参照文書を結果に含めるか（既定は False）
        }
    },
    "track_total_hits": True,  # 総件数を正確計上（評価時に便利）
    # ,"size": 10             # 返す件数を制御（既定: 10）
}

# --- 推奨: ES 7/8 互換の like（_type を削除） -------------------------------
# query = {
#     "query": {
#         "more_like_this": {
#             "fields": ["text"],
#             "like": [{ "_index": jp_index, "_id": "3" }],
#             "min_term_freq": 1,
#             "min_doc_freq": 1,
#             "max_query_terms": 50,
#             "minimum_should_match": "30%"
#         }
#     },
#     "track_total_hits": True,
#     "size": 10
# }
# -----------------------------------------------------------------------------

# 検索実行
res = es.search(index=jp_index, body=query)

# 結果表示
# - _score は BM25 による関連度。高いほど参照文書に“似ている”。
# - 参照文書そのものは（通常は）除外されるが、設定や ES のバージョンによっては残ることがある。
w1 = res["hits"]["hits"]

for item in w1:
    score = item["_score"]
    source = item["_source"]
    app_id = source["app_id"]
    title = source["title"]
    print(app_id, title, score)

# 参考: デバッグ/可観測性向上の補助（必要に応じて活用）
# - 代表語の確認（MLT がどの語をクエリ化したかは API では直接取れないため、
#   近似として参照文書 text を analyze API で可視化し、tf/df の高低を観察する）
# toks = es.indices.analyze(index=jp_index, body={"analyzer": "jpn-search", "text": w1[0]["_source"]["text"][:2000]})
# for t in toks["tokens"][:50]:
#     print(t["token"], t["position"])